S1: What OpenClaw is

OpenClaw is an FOSS, self-hosted AI gateway and personal assistant framework. 
Think of it as a local WebSocket server that sits in the middle between:
   - Messaging Platforms
   - AI agents

It gives agents the ability to:
   - Receive and send messages across any chat platform
   - Execute tools (browser automation, bash commands, file operations, memory
     search)
   - Manage sessions and long-term memory
   - Run inside sandboxed Docker containers


WHY THIS MATTERS FOR YOUR PROJECT: OpenClaw is essentially a working blueprint
for giving an AI agent the infrastructure to operate semi-autonomously across
multiple channels. It has the tool system, session persistence, multi-provider
support, and plugin architecture that your "New Dejima" agents would need.



---
SECTION 2: THE ENTRY POINT & HOW IT WORKS

The boot chain is simple:
   openclaw.mjs -> dist/entry.js -> src/index.ts -> CLI program (Commander.js)

- openclaw.mjs - The file you run. It just loads the compiled distribution.
- src/index.ts - Builds the CLI command tree using Commander.js with 100+
  command handlers.
- The main command is `gateway` which starts the WebSocket control plane on port
  18789.


WHY THIS MATTERS FOR YOU: Your agent loop system needs an entry point like this.
When an agent "Wakes up", something needs to bootstrap the environment, load
tools, and start the main loop. OpenClaw's boot chain is a clean reference for
that.



---
SECTION 3: THE GATEWAY (`src/gateway/`, ~130 files)

This is the BRAIN of OpenClaw. The Gateway is a WebSocket server that handles:
   - SESSION MANAGEMENT - Each conversation gets an isolated session (main
     sessions for 1-on-1, group sessions for groups)
   - CHANNEL ROUTING - Incoming messages get routed to the right agent based on
     rules.
   - CLIENT MANAGEMENT - WebSocket connection lifecycle, authentication.
   - CONFIGURATION - Config reloading, state persistence
   - PRESENCE & TYPING - Real-time status indicators
   - AGENT INTEGRATION - Connects to the Pi agent via RPC.

WHY THIS MATTERS: Your agent consciousness loop needs soemthing like this 
   Gateway. it's the central control plane that coordinates everything. Your 
   agent needs a persistent process that manages their state, routes their 
   actions, and coordinates their tool usage.


---   
SECTION 4: THE AGENT RUNTIME (`src/agents/`, ~300+ files)

This is where agents actually THINK AND ACT:
   - Pi Embedded Runner (`pi-embedded-runner.ts`) - Runs agent inference 
        in-process, manages session state
   - Pi Embedded Subscribe (`pi-embedded-subscribe.ts`) - Streams responses with
        tool events
   - Model Selection (`model-selection.ts`, `model-fallback.ts`) - Multi-model
        failover
   - Auth Profiles - OAuth token rotation across providers
   - Tool Management - 60+ tool definitions (browser, bash, canvas)
   - Skills - Workspace/managed/bundled skill loading and execution
   - Subagents - Multi-agent routing and cross-agent message spawning
   - System Prompts - Dynamic prompt building per agent/session


CRITICAL FOR YOUR PROJECT:   
   This is the core of what you need to replicate. The agent runtime handles:
      1. Making LLM API calls with streaming
      2. Tool execution (the agent calling bash, browsing web, etc.)
      3. Multi-model failover (if one provider fails, try another)
      4. Subagent spwaning (one agent creating another - your "reproduction"
         system)




---
SECTION 5: THE TOOL SYSTEM

OpenClaw gives agents 60+ BUILT-IN TOOLS:
   - BROWSER - Playwright-based web automation
   - BASH - Shell command execution
   - CANVAS - Interactive UI rendering
   - MEMORY SEARCH - Vector search over past conversations
   - FILE OPERATIONS - Read, write, edit files
   - SESSION MANAGEMENT - Create/switch between sessions

Tools have a POLICY ENFORCEMENT layer - per-agent and per-session restrictions
on what tools an agent can use. There's also an APPROVAL WORKFLOW where 
dangerous commands require human approval.

CRITICAL FOR YOUR PROJECT: Your agents need tools to make money. OpenClaw's
tool system shows you how to:
   - Define tools with schemas
   - Execute them safely in sandboxes
   - Enforce policies on what agents can/can't do
   - Track tool calls with IDs for auditing

In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣀⣠⣤⣤⣤⣤⣤⣤⣄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⣠⠾⠛⠉⠉⠉⠁⠀⠀⠉⠉⠉⠉⠛⠷⣄⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢸⡏⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⡇⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⢸⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣸⠇⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠹⣧⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣾⠏⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⣄⠀⠀⠀⠀⠀⠀⠀⠀⢠⡿⠃⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢿⡄⠀⠀⠀⠀⠀⠀⢀⡿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣷⡀⠀⠀⠀⠀⢀⣾⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣷⡀⢀⠀⢀⣾⠇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣗⠒⠨⢥⣾⣦⠬⠑⠒⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣸⣤⣄⣸⠏⢿⣀⣤⣴⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⢻⣿⣍⡉⠁⠉⣉⣽⣿⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡸⢀⣼⣁⣠⣀⣹⣄⠘⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⠷⢾⣿⠿⠉⠹⢿⣿⡶⠷⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠀⠀⠀⠉⠁⠀⠀⠀⠀


---   
SECTION 6: THE PLUGIN & EXTENSION SYSTEM (`extension/`, 30+ plugins)

OpenClaw uses a plugin architecture for extensibility:
   - Each plugin lives in `extensions/` as its own workspace package
   - Has a manifest file `openclaw.plugin.json` defining its ID, channels, and
     config schema
   - Plugins are lazy-loaded and validated with Zod schemas.
   - They can provide: new channels, auth providers, tool slots, hooks


EXISTING PLUGINS INCLUDE: Discord, Telegram, Slack, Signal, iMessage...

WHY THIS MATTERS: Your "New Dejima port system" - the interfaces that let agents
   interact with the real world - this IS the plugin system. Each new 
   marketplace, exchange, or service you want agents to access becomes a plugin/
   extension.




---   
SECTION 7: MEMORY & SEARCH (`src/memory/`)

OpenClaw has a sophisticated long-term memory system:
   - QMD Manager        - 

In [ ]:
⣤⣤⣤⣤⣤⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⣿⣿⠭⠏⠉⣷⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀M
⣿⣿⡁⠠⢀⠉⠉⣷⣶⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀I
⠀⢰⣿⡇⠀⡄⠁⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀N
⠀⠀⠉⢹⣿⡇⠀⡁⠂⠉⢹⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀ E
⠀⠀⠀⠀⠀⠿⠿⣅⣀⠉⠂⠀⠿⣿⣁⣀⠀⠀⠀⠀⠀⢀⣀⣀⣀⠀⠀⠀⠀C
⠀⠀⠀⠀⠀⠀⠀⠿⠿⣀⣌⠐⠀⡀⠿⠿⣀⣀⠀⢀⣀⣿⣿⣿⣿⠀⠀⠀⠀R
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠿⢷⣄⡂⠄⠠⠀⠿⢿⣀⣼⠿⣿⣿⠿⠿⠀⠀⠀⠀A
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛⣧⣤⡆⠁⠄⢸⣿⣏⢣⣿⣿⠀⠀⠀⠀⠀⠀F
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⣷⣶⣾⣾⡛⣿⣶⡏⠉⠀⠀⠀⠀⠀⠀T
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣲⣶⣿⣿⠹⣿⣿⣿⣿⣷⣶⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢠⣤⣿⣿⣿⣿⣿⣿⠛⢻⣿⣿⣿⣤⣤⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⣿⣿⡋⠉⠉⠉⠀⠈⠉⢿⣿⣿⣿⣶⣶⣶⡆
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⣿⣽⣿⣿⠁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡿⠿⢿⣿⣿⠇

OpenClaw is an FOSS, self-hosted AI gateway and personal assistant framework.
Think of it as a local WebSocket server that sits in middle between:
   - Messaging Platforms
   - AI Agents


It gives AI agents the ability to:   
- Receive and send messages across any chat platform
- Execute tools (browser automation, bash commands, file operations, memory
  search)
- Manage sessions and long-term memory
- Run inside sandboxed Docker containers


WHY THIS MATTERS FOR YOUR PROJECT: OpenClaw is essentially a working blueprint
for giving an AI agent the infrastructure to operate semi-autonomously across
multiple channels. It has the tool system, session persistence, multi-provider
support, and plugin architecture that your "New Dejima" agents would need.




<<<<>>>>
SECTION 2: THE ENTRY POINT & HOW IT BOOTS

The boot chain is simple:

   openclaw.mjs -> dist/entry.js -> src/index.ts -> CLI program (Commander.js)

- openclaw.mjs - The file you run. It just loads the compiled distribution.
- src/index.ts - Builds the CLI command tree using Commander.js with 100+ 
  command handlers.              <-- wow! all the handlers!!!
- The main command in `gateway` which starts the WebSocket control plane on port
  18789.

WHY THIS MATTERS FOR YOU: Your agent loop system needs an entry point like this.
   When an agent "wakes up", something needs to bootstrap the environment, load
   tools, and start the main loop. OpenClaw's boot chain is a clean reference
   for that.




<<<<>>>>   
SECTION 3: THE GATEWAY (`src/gateway/`, ~130 files)

This is the BRAIN of OpenClaw, The Gateway is a WebSocket server that handles
(main):




   

In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢞⡗⠪⡷⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⣄⠀⠀⠈⡝⠃⡃⠀⠀⢀⣐⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣋⢰⡆⠄⣰⡾⡒⣲⡥⣻⣙⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠟⠼⠊⠘⢱⢛⠏⠈⠛⠹⣝⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠁⠀⠀⢀⡜⢧⡧⠀⠀⠀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣫⢭⡻⣙⠣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⡀⡞⡍⣥⠟⣦⡤⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⢄⡶⢷⢸⠧⢙⠻⡰⡟⡿⠎⣷⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡀⢤⢄⢀⡀⣸⢾⢳⠂⣠⡓⣽⣦⣻⠖⡗⣤⠗⡷⣞⣻⣢⣤⣄⠄⣀⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⢀⡤⣦⡴⣔⠥⠟⠓⠙⠘⠱⢴⢮⢩⡼⠨⢙⡶⡄⣬⣸⡡⠃⣌⡫⣬⣗⢹⣦⣱⡘⠋⢙⠿⡸⡽⣲⡠⢤⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⢀⣠⣠⣏⠺⠛⠘⠀⠀⠀⠀⠀⠀⢴⣤⣾⢯⢮⡬⢧⣛⣾⢁⡹⢹⡥⣸⡻⢧⢫⣋⣬⣷⣩⢷⠀⠀⠀⠈⠀⠋⠍⡶⢏⣓⣢⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢴⢟⣱⣴⠻⠀⠀⠀⠀⠀⠀⠀⠀⠀⡱⣟⡖⣽⣶⢆⣶⣦⣡⢨⣄⣠⡝⠃⣦⣆⣠⣫⣆⢸⣿⣾⡁⠀⠀⠀⠀⠀⠀⠈⠑⡞⡧⣚⢷⣆⡀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠐⢗⢸⡳⢿⣁⣇⡀⠀⠀⠀⠀⠀⠀⠀⠑⢺⡇⡟⣙⣔⣿⣻⡍⠿⣬⣗⣇⠟⣌⣟⣂⡟⢿⡞⡿⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⣞⡽⠇⢽⠆⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠈⠳⢟⡼⣽⣻⣱⣦⣄⣀⠀⠀⠀⠀⠀⠈⠿⡾⡋⣟⢷⣻⣟⡿⣳⣟⣝⣯⡳⣼⣧⡻⡿⣽⠃⠀⠀⠀⠀⠀⠀⢀⢀⡴⣾⣿⡮⣺⣡⠟⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠘⠚⠵⣆⣿⡴⠾⢴⣦⢄⣤⣤⣼⣞⣈⡉⠚⢙⣃⠒⠛⠃⢚⡛⠻⣨⣼⠾⢇⣅⣀⣀⡤⣄⣔⣶⣎⡳⡣⣽⡣⠷⠛⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠙⠛⠻⠮⠯⢽⣇⣚⣿⣯⠿⡹⣿⣿⡯⢻⣮⣻⢏⡻⣍⣸⣱⣭⣍⣭⡷⠯⠵⠛⠚⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⠉⠻⠛⠛⠿⠿⢒⠻⠟⠉⠉⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

---

- I believe it would be useful to ask Claude to explain to me the key functionalities
  of each of the file's functionalities... as in... explain the files that
  are top 5 or top 10 important for each section that's related to my current
  project... and then i will just go through them
     - tbh we can also share in the plans too... of how we want to be the
       super-architect... and i guess it is hence from here, that it would be
       incredibly important for us to know what files to change what, and where
     - come to think of it... would it be useful to fill up an additional 
       sub-directory which consist of a skeleton repository... like a tree
       directory... that's a direct one-to-one copy of the original openclaw repo
       but instead of having modified code... we first only have and is filled with
       md files that plans on what we want to implement... and why are these
       certain files useful, etc.
          - in fact we need to start doing this for android project already...
            

In [ ]:
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢰⣶⣶⣶⣶⣶⣶⣶⣶⣶⣶⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⢰⣶⡎⠉⠉⠉⠉⠉⠉⠉⠉⠉⠉⢱⣶⣶⣶⣶⣶⣶⡆⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠘⠛⢣⣤⣤⣤⣤⣤⣤⣤⣤⡄⠀⠈⠛⠛⠛⠛⣿⣿⡇⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠛⠛⠛⠛⠛⠛⠛⠛⢣⣤⡄⠀⠀⠀⠀⣿⣿⡇⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⡸⠿⢇⣀⣀⠀⠀⠻⠿⢃⣀⡀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣀⣀⠿⠿⢇⣀⡸⠿⠿⣀⣀⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⣿⣿⡇⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⡇⠀⠀⣿⣿⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⠀⠀⢰⣶⡎⠉⠱⣶⣶⠉⠉⠀⠀⠀⠀⠀⠀⠀⣿⣿⠀⠀⢸⣿⡇
⠀⠀⠀⠀⠀⣶⣶⡌⠉⢱⣶⡆⠉⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠉⠉⣴⣶⡎⠉⠁
⠀⠀⠀⣤⣤⠛⠛⢡⣤⡜⠛⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠛⠛⠃⠀⠀
⢀⣤⣤⠛⠛⣤⣤⡘⠛⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⢸⣿⣿⣀⣀⠿⠿⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠸⠿⠿⠿⠿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

---

# PLANS

We can instead try and be the super-architect... by asking Opus WHICH DIRECTORIES
AND IN WHICH EXISTING FILES inside the openclaw/openclaw repository... that we
should introduce changes
   - and for each file, add what features
   - and where should each feature of dejima be added





- look up $BUNKER, and ask Claude Code to analyse it... and ask it what portion
  of $Bunker can we use
     "/Users/administrator/Imperial-College-London/Projects/2026/2026-02 February/Project Altiera/resources/moltbunker-sdk"



- look up Grok 4.20 SITUATIONAL-AWARENESS
   - we want to analyse what methods were given to achieve such incredible rates
     of returns... if this system can be continuously demonstrated to be 
     profitable... then this system is already viable...   

In [ ]:
⠀⠀⠀⠀⠀⠀⡶⠒⠒⠒⠒⢒⣦⡀⠀⠀⠀⠀⣶⠒⠒⠒⠒⠒⣶⣶⠒⠒⠒⠒⠒⣶⣶⠒⠒⠒⠒⠒⣶⠀⢀⡖⠒⠒⠒⠒⢲⣶⡶⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⣶⡖⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⣶⡖⠒⠒⠒⠒⠒⠒⠒⠒⠓⠒⠒⢲⣶⡖⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠒⠲⣶⣶⠒⠚⠛⠚⠚⠳⢖⠒⠒⠒⠒⠒⣶⣶⠒⠒⠒⠒⠚⠚⠛⠓⠒⡖⠒⠒⢶⠀⠀
⠀⠀⠀⠀⠀⣸⠁⠀⠀⠀⠀⣼⣿⣷⡄⢀⣀⣰⡃⠀⠀⠀⠀⢰⣿⡇⠀⠀⠀⠀⢰⣿⡇⠀⠀⠀⠀⢀⣿⣇⢸⣇⠀⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣿⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⣿⣿⠀⠀⠀⣀⣀⠀⠀⣀⣀⡀⠀⠀⢿⣿⡀⠀⠀⠀⠀⠀⠀⠑⡄⠀⠀⠀⢹⣿⡆⠀⠀⠀⠀⠀⠀⠀⠀⣷⠀⠀⠘⡆⠀
⠀⠀⠀⠀⢠⠇⠀⠀⠀⠀⠀⠀⢸⣿⣿⣾⠃⠀⠀⠀⠀⠀⠀⣾⣿⠀⠀⠀⠀⠀⣸⣿⠁⠀⠀⠀⠀⠀⠀⣼⣿⠁⠀⠀⠀⠀⣹⣿⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿⣷⣀⣀⡄⠀⠀⣿⣿⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠿⡿⣇⣀⡽⠿⠇⠀⠀⢸⣿⡇⠀⠀⠀⠀⠘⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡆⠀⠀⠀⠀⠸⣿⣿⡿⠁⠀
⠀⠀⠀⠀⡞⠀⠀⠀⠀⠀⠀⠀⠛⠛⠛⠛⠒⠒⠒⠒⠓⠒⢲⣿⡇⠀⠀⠀⠀⢀⣿⡿⠀⠀⠀⠀⠀⠀⠀⠛⠛⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⠘⠛⠛⢻⠛⠛⠛⢻⣿⡇⠀⠀⠀⠀⠲⣿⣿⣿⣿⣿⣿⣿⣿⣿⠀⠀⠈⠁⠀⠛⠛⠀⠀⠀⢠⣤⣼⣿⡇⠀⠀⠀⢹⣿⣿⣿⣶⠀⠀⠀⠈⣿⣷⠀⠀⠀⠀⠀⠛⠛⠛⠛⡛⠛⠛⢻⣿⣿⣿⣿⣷⠀⠀⠀⠀⠀⢻⡋⠀⠀⠀
⠀⠀⠀⣸⠃⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⣼⣿⠁⠀⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⢠⣶⡞⠀⠀⠀⠀⠀⠀⠀⣿⡟⠀⠀⠀⠀⠀⠀⠀⠀⠘⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠉⢹⣿⡇⠀⠀⠀⠘⠟⠀⠉⠛⠂⠀⠀⠀⢻⣿⡄⠀⠀⠀⠀⠀⠀⠀⠀⡇⠀⠀⠈⣿⣿⣿⣿⣿⡇⠀⠀⠀⠀⠘⣇⠀⠀⠀
⠀⠀⢠⡏⠀⠀⠀⠀⢀⣿⡿⠀⠀⠀⣸⣿⠇⠀⠀⠀⠀⢠⣿⡏⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⣸⣿⡇⠀⠀⠀⠀⡀⠀⢸⣿⡇⠀⠀⠀⠀⠀⣿⣿⣿⣿⣿⣿⣿⣿⣿⠇⠀⠀⠀⠀⢠⡇⠀⠀⠀⠀⣿⡏⣿⣿⠀⠀⢀⡀⠀⣸⣿⡇⠀⠀⠀⠀⠈⣿⣷⠀⠀⠀⠀⠀⣀⣀⠀⠀⠀⠀⠀⠸⣿⡇⠀⠀⠀⠀⠈⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣀⠀⠀⠀⠀⢹⡄⠀⠀
⠀⠀⡾⠀⠀⠀⠀⠠⣾⣿⣅⣀⣀⣠⣿⡿⠀⠀⠀⠀⠀⣼⣿⠃⠀⠀⠀⠀⢰⣿⡏⠀⠀⠀⠀⠈⣿⣿⣿⣿⠀⠀⠀⡈⠀⢸⣿⡇⠀⠀⠀⠀⠀⠿⠿⠿⠿⠿⢿⠿⣿⣿⠀⠀⠀⠀⠀⠘⠷⠶⠶⠶⠶⠿⠿⣿⣿⠀⢸⠀⠀⠀⢸⣿⡇⠀⠀⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⢸⣿⡗⠒⠀⠠⠄⠤⣿⣿⠀⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⠟⣿⣿⣿⣿⣿⣿⡏⠁⠀⠀⠀⠀⢷⠀⠀
⠀⢸⠇⠀⠐⠀⠀⢠⣿⣿⣿⣿⣿⣿⣿⠃⠀⠀⠀⠀⢀⣿⡟⠀⠀⠀⠀⠀⣸⣿⠃⠀⠀⠀⠀⢀⣿⣿⣿⡏⠀⠀⠀⠇⠀⣾⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⡔⠁⠁⠀⣿⣿⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢂⠀⠀⢿⣿⡀⡸⠀⠀⠀⠘⣿⣧⠀⠀⠀⠀⠀⢻⣿⡄⠀⠀⠀⠀⠈⣿⣧⠀⠀⠀⠀⠀⢹⣿⡆⠀⠀⠀⠀⠈⣿⣿⣿⣿⣿⡆⢹⣿⣿⣿⣿⣿⣿⡀⠀⠀⠀⠀⠘⡆⠀
⠀⢿⣦⣤⣤⣤⣴⣾⣿⣿⣿⣿⣿⣿⣿⣦⣤⣤⣤⣴⣾⣿⣷⣤⡾⣦⣤⣴⣿⣿⣦⣴⣶⣤⣤⣾⣿⣿⣿⣷⣤⣦⣶⣶⣴⣿⣿⣦⣤⢾⣦⣶⣦⣶⣶⣶⣷⣶⣶⣶⣿⣿⣦⣤⣴⣰⣿⣶⣤⣤⣤⣴⣾⣤⣴⣾⣿⣷⣷⣴⣶⣦⣶⣿⣿⣦⣴⢢⣤⣤⣾⣿⣷⣤⣴⣶⣤⣴⣿⣿⣶⣶⣶⣶⣴⣾⣿⣷⣶⣤⣶⣦⣶⣿⠋⢿⣿⣿⣷⠀⣿⣿⣿⣿⣿⣿⣷⣶⣤⣤⣴⣴⠟⠀
⠀⠀⣹⢿⣿⣿⣿⣛⢻⣿⣿⣿⣿⡟⢻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣦⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣇⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⡛⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣛⣛⣿⣿⣿⡟⠁⠀⠸⣿⣿⣿⡆⢸⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣇⠀⠀
⠀⢠⡟⠀⠙⢿⣿⣿⣿⣿⣿⣿⣿⠁⣾⣿⠉⠻⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠁⢻⣿⣶⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣶⣾⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⣿⠏⠀⠀⠀⠀⢻⣿⣿⣧⠀⣿⣿⣿⣿⣿⣿⣿⡿⠋⠀⢻⡀⠀
⠀⣼⠁⠀⠀⠀⠈⠉⠉⠉⠉⠉⠁⢠⣿⡏⠀⠀⠈⠉⠉⠉⠉⠉⠉⠈⠉⣹⣿⠉⠉⠉⠈⠉⠉⠉⠉⠉⠩⠀⢸⣿⡏⠉⠉⠉⠉⠁⠉⠉⠉⠉⠉⠉⣽⣿⠋⠉⠉⠉⠉⣿⡏⠉⠉⠉⠉⠉⡏⠀⠀⠀⣿⣿⡏⠉⠉⠉⠉⢹⣿⣿⣿⣿⣿⣿⡏⠉⠉⠁⠉⢹⣿⡍⠉⠉⠁⠉⠉⠉⠉⠉⠉⠁⠉⣿⣏⠉⠉⠉⠁⠀⠀⠀⠀⠀⠘⣿⣿⣿⡄⠈⠉⠉⠉⠉⠉⠉⠀⠀⠀⠈⣧⠀
⢰⡏⠀⠀⢀⣄⢀⡠⠤⠠⠤⠤⠤⣼⣿⠁⠀⠀⠀⠀⠠⠤⠤⠤⠤⠤⣴⣿⡏⢀⣀⠤⠤⠤⠤⣤⡤⢤⣆⡠⣼⣿⡇⠀⠀⠀⠀⠤⢤⠤⠤⠠⠤⠤⣿⣿⠀⠀⡄⠀⢤⣿⡏⠀⠀⠀⠀⢀⡇⠀⠀⠀⣿⣿⣿⠀⠀⠀⠀⢸⣿⣿⣿⣿⣿⣿⣇⠀⢠⣄⡠⢼⣿⡇⢀⠀⠀⠤⠤⠤⠤⠤⠠⠤⠤⣿⣿⣀⣀⢤⣄⣀⡠⠤⠤⠤⠤⢿⣿⣿⣧⡀⠀⠤⠤⠤⠄⠀⢀⠀⠀⠀⢹⡄
⠘⠷⣄⠀⠀⠑⠀⠀⠀⠀⠀⠀⠀⠈⠻⣦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠹⣷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠃⠀⠈⢿⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢹⣿⠀⠀⠁⠀⠀⣿⣇⠀⠀⠀⠀⠀⡇⠀⠀⠀⣿⣿⡇⠀⠀⠀⠀⢠⣿⣿⣿⣿⣿⣿⠃⠀⠀⠁⠀⣸⡿⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⣾⠏⠀⠀⠀⠃⠀⠀⠀⠀⠀⠠⢾⣿⠟⠁⠀⠀⠀⠀⠀⠀⠀⠜⠀⠀⣠⠞⠃
⠀⠀⠈⠳⣤⡀⠀⠀⠀⠀⢤⣤⣤⣤⣤⣬⣿⣦⣤⣤⣤⡀⠀⠀⠀⠀⢤⣤⣼⣿⣆⠀⠀⠀⠀⠠⣤⡀⠀⠀⠀⠈⢿⡆⠀⠀⠀⠀⢠⣤⠀⠀⠀⠀⠈⣿⡇⠀⠀⠀⠀⢸⣿⠀⠀⠀⢀⣠⡇⠀⠀⠀⣿⣿⠁⠀⠀⠀⠀⠚⢻⣿⣿⡿⠛⠋⠀⠀⠀⠀⣰⡿⠁⠀⠀⠀⢀⣤⠆⠀⠀⠀⠀⣰⡿⠁⠀⠀⠀⢀⣤⠆⠀⠀⠀⠀⣴⠟⠁⠀⠀⠀⣠⣤⣤⣤⣤⣤⣤⡴⠞⠁⠀⠀
⠀⠀⠀⠀⠈⠙⢦⣀⠀⠀⠀⠙⠿⠿⠿⠿⠿⠿⢿⣿⣿⣷⣄⠀⠀⠀⠈⠻⣿⣿⣿⣿⡉⠀⠀⠀⠙⣷⡄⠀⠀⠀⠈⣿⡄⠀⠀⠀⠀⠻⠧⠀⠀⠀⣀⣻⣷⠀⠀⠀⠀⠸⠿⠀⠀⠀⠀⠀⡇⠀⠀⠀⣿⣿⠀⠀⠀⠀⠀⠉⠿⠿⠿⠇⠀⠀⠀⠀⠀⢠⣿⠁⠀⠀⠀⢤⣿⠏⠀⠀⠀⢀⣾⠟⠀⠀⠀⢀⣴⡟⠁⠀⠀⢀⣴⡾⠋⠀⠀⠈⠩⠿⠿⠿⠿⠿⣿⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠙⠷⣆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢿⣿⣿⣷⣄⠀⠀⠀⠙⢿⣿⣿⣷⡄⠀⠀⠀⠘⣿⡄⠀⠀⠀⠘⣿⡄⠀⠀⠀⠀⢀⡀⠀⠀⠙⠻⣿⣦⠤⢴⡄⠀⠀⠀⢠⣤⠤⠼⠇⠀⠀⠀⣿⡏⠀⠀⠀⠀⠀⡀⠀⠀⠀⣀⠀⠀⠀⠀⢠⣿⠃⠀⠀⠀⢠⣿⠃⠀⠀⠀⣠⣿⠃⠀⠀⠀⣠⡾⠋⠀⠀⠀⣠⡾⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⣠⠞⠋⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⠛⣿⣿⣿⣿⣷⣄⠀⠀⠀⠙⢿⣿⣿⣧⡀⠀⠀⠈⠻⣿⣿⣿⣆⠀⠀⠀⠈⢿⣄⠀⠀⠀⠘⣿⡀⠀⠀⢀⣹⣷⠀⠀⠀⠀⢻⣿⠀⠸⡇⠀⠀⠀⠀⣿⠀⠀⠀⠀⠀⠀⣿⡇⠀⠀⠀⢰⣿⠃⠀⢀⣾⡏⠀⠀⠀⢠⣿⠃⠀⠀⠀⣰⡿⠁⠀⠀⠀⣼⡟⠁⠀⠀⢠⣾⠏⠀⠀⠀⣠⡾⠋⠀⠀⢀⣠⣾⣿⣿⣿⣟⠛⠛⠁⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢧⣀⠀⠀⠀⠁⠀⠀⠀⠉⠛⣿⡏⠙⢦⡀⠀⠀⠙⢿⡇⠘⢷⡀⠀⠀⠀⠀⠀⠀⠀⠀⠘⣷⡀⠀⠈⠉⢿⣧⠀⠀⠀⠸⡏⠀⠀⣿⠀⠀⡀⠀⣿⠀⠀⠀⠀⠀⠀⣿⠁⠀⠀⠀⣼⠛⠛⠛⢻⡿⠀⠀⠀⢀⣿⠃⠀⠀⠀⠀⠀⠀⠀⠀⢀⣾⠋⠉⠉⠀⠀⠀⠀⠀⣠⣤⡾⠋⠀⠀⠀⡀⠀⠀⠀⠀⣠⡴⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠁⠀⠈⠛⠛⠛⠛⠛⠃⠀⠀⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠃⠀⠀⠙⠛⠛⠛⠛⠃⠀⠀⠀⠀⠀⠀⠙⠛⠛⠛⠛⠋⠀⠀⠀⠘⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠁⠘⠛⠛⠛⠛⠛⠛⠛⠛⠛⠛⠋⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⢄⣠⠤⠤⣄⡐⣢⠥⠤⢤⣀⡖⠉⠉⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡟⢠⠞⢳⡄⢻⢁⡔⠻⣦⠙⣗⣦⣴⣧⡤⣤⡀⠀⠀⢠⡤⠤⠤⣄⣀⣀⣀⡴⣤⡴⣄⣦⢦⠠⡀⠀⣄⡴⣄⡀⠀⠀⠀⠀⢀⠔⣤⠦⠤⣄⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⡊⠀⠙⣄⡤⠤⠤⡀⡀⠀⠀⠀⠀⡀⢠⢦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢧⠘⠧⡽⢏⣼⡈⠳⡬⠟⣠⢯⠃⢻⠏⠶⠞⢹⠀⠀⢘⣶⡆⣶⣾⢟⣛⡻⡇⢸⡇⣿⡋⠘⣳⡖⣛⣾⡇⣿⡴⣒⢒⢦⡀⠈⢸⠉⡖⠒⢻⡖⡒⡲⡖⢒⣖⢦⡶⢒⣒⢦⢖⣒⢒⣷⡦⠒⢿⡀⠒⠲⡿⠚⢒⢶⠒⢖⠒⠿⢺⠗⠓⡲⣤⠖⣒⢒⡆⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛⢲⡖⢋⠭⢉⠲⡶⡚⠏⠂⠀⣼⠀⣾⡆⢸⠀⠀⠀⢿⡇⡟⢻⡠⠬⠤⣇⠸⣇⢿⣿⡘⢟⡣⠤⠀⣇⢿⣄⠤⠥⢼⡇⠐⢸⡀⠿⢇⣸⡡⠤⠄⡇⢠⣼⠸⣀⠤⠤⢼⠦⠤⢝⡯⠇⠈⣞⠹⠿⢀⣇⠤⠤⢼⠀⣾⣷⠆⢸⡀⠤⠤⡾⠦⠤⢙⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡀⢷⠈⢸⠆⣹⠇⠀⠀⠀⠈⠛⠁⠙⠋⠀⠀⠀⠀⠙⠁⠈⠉⠚⠚⠉⠛⠉⠋⠁⠑⠉⠑⠛⠛⠉⠛⠉⠛⠓⠋⠁⠀⠁⠉⠒⠊⠁⠙⠛⠚⠛⠁⠈⠊⠉⠛⠒⠛⠓⠒⠉⠀⠀⠀⠈⠉⠒⠉⠈⠑⠓⠊⠙⠁⠃⠈⠋⠉⠒⠒⠋⠓⠒⠉⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣤⣉⣡⠴⡣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀

---

- WebSockets are a computer communication protocol providing full-duplex,
  bidirectional communication channels over a single, long-lived TCP connection.
  Unlike HTTP's request-response model, WebSockets allow servers to push data
  to clients insantly, making them ideal for low-latency, real-time applications
  like chat, gaming and live financial feeds.

- RPC (Remote Procedure Call) is a client-server networking protocol that allows
  a program to execute code on a remote server as if it were a local subroutine,
  withut needing to understand network details. It uses stubs, serialisation
  (marshalling), and protocols like TCP or HTTP to facilitate communication,
  commonly used in distributed systems, microservices, and API design.  




- Playwright is an open-source framework for cross-browser automation and 
  end-to-end web application testing. It was designed to be a fast, reliable, 
  robust, and evergreen test automation framework, and its API supports modern
  rendring engines that include Chromium, Firefox, WebKit, etc. ...
  



- A Zod schema is a TypeScript-first validation library used to define, validate
  , and parse data structures, ranging from simple strings to complex nested
  objects. It allows developers to declare a schema once and automatically
  infer static TypeScript types, eliminating duplicate code and ensuring runtime
  type safety.  